# Binary Bayes Net Inference

This is a quick notebook exercise to exemplify Bayes Net (BN) inference. 

Consider the following BN:  

![Imaginary SuperBowl Bayes Net Diagram](BN-NFL.png "Imaginary SuperBowl Bayes Net Diagram")


----
We can use the `BayesianNetwork` module from [pgmpy](https://pgmpy.org/) to construct this network:

In [9]:
import numpy as np
import pandas as pd

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

/Users/ronitp/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 3.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 2.8 MB/s eta 0:00:0000:0100:01
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl (417 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 3.3 MB

In [10]:
# Define Bayesian Network structure
model = BayesianNetwork([('F', 'Q'), ('F', 'D'), ('Q', 'W'), ('D', 'W')])

# Define CPDs
cpd_f = TabularCPD(variable='F', variable_card=2, values=[[0.3], [0.7]], state_names={"F":["low", "high"]})
cpd_q = TabularCPD(variable='Q', variable_card=2, values=[[0.5, 0.2], [0.5, 0.8]],
                    evidence=['F'], evidence_card=[2], state_names={"F":["low", "high"], "Q": ["bad", "good"]})
cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.6, 0.3], [0.4, 0.7]],
                    evidence=['F'], evidence_card=[2], state_names={"F":["low", "high"], "D": ["weak", "strong"]})
cpd_w = TabularCPD(variable='W', variable_card=2, 
                    values=[[0.30, 0.5, 0.15, 0.25], [0.70, 0.5, 0.85, 0.75]],
                    evidence=['Q', 'D'], evidence_card=[2, 2], state_names={"Q":["bad", "good"], "D": ["weak", "strong"], "W": ["lose", "win"]})

# Add CPDs to model
model.add_cpds(cpd_f, cpd_q, cpd_d, cpd_w)

# Check Model
assert model.check_model()

In [11]:
_ = [print (cpd) for cpd in model.get_cpds()]

+---------+-----+
| F(low)  | 0.3 |
+---------+-----+
| F(high) | 0.7 |
+---------+-----+
+---------+--------+---------+
| F       | F(low) | F(high) |
+---------+--------+---------+
| Q(bad)  | 0.5    | 0.2     |
+---------+--------+---------+
| Q(good) | 0.5    | 0.8     |
+---------+--------+---------+
+-----------+--------+---------+
| F         | F(low) | F(high) |
+-----------+--------+---------+
| D(weak)   | 0.6    | 0.3     |
+-----------+--------+---------+
| D(strong) | 0.4    | 0.7     |
+-----------+--------+---------+
+---------+---------+-----------+---------+-----------+
| Q       | Q(bad)  | Q(bad)    | Q(good) | Q(good)   |
+---------+---------+-----------+---------+-----------+
| D       | D(weak) | D(strong) | D(weak) | D(strong) |
+---------+---------+-----------+---------+-----------+
| W(lose) | 0.3     | 0.5       | 0.15    | 0.25      |
+---------+---------+-----------+---------+-----------+
| W(win)  | 0.7     | 0.5       | 0.85    | 0.75      |
+---------+---

----
Calculate $P(W|F=\text{high})$

$$
\begin{align}
P(W|F=\text{high}) & = \\
& \propto P(W,F=\text{high}) \\
& = \sum_{q\in Q,d \in D} P(F=\text{high}, Q, D, W) \\
& = \sum_{q,d} P(F=\text{high})P(q|F=\text{high})P(d|F=\text{high})P(W|q,d) 
\end{align}
$$

In [12]:
P_w = None

P_w = (0.7 * 0.2 * 0.3 * 0.7) + (0.7 * 0.2 * 0.7 * 0.5) + (0.7 * 0.8 * 0.3 * 0.85) + (0.7 * 0.8 * 0.7 * 0.75)

P_w

# Calcuate the probabiity of winning and losing 
# and put it in a the P_w variable

# YOUR CODE HERE
# raise NotImplementedError()

# P_w = P(Chiefs Win | F = High)

0.5152

In [ ]:
# This cell intentionaly left empty


----
Then we can use Variable Elimination to do the same inference. 

Variable Elimination is based on the following insight:

$$
\begin{align}
& \sum_{q,d} P(F=\text{high})P(q|F=\text{high})P(d|F=\text{high})P(W|q,d) \\
& = P(F=\text{high}) \sum_{q,d} P(q|F=\text{high})P(d|F=\text{high})P(W|q,d) \\
& = P(F=\text{high}) \sum_{q} P(q|F=\text{high})\sum_{d}P(d|F=\text{high})P(W|q,d) \\
\end{align}
$$

----

Now use the `VariabeElimination` functionalityin `pgmpy` to calcuate the same probability.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

----
Here's a more complex example, using the indurance BN:

In [ ]:
from pgmpy.utils import get_example_model

model = get_example_model('insurance')
model.get_cardinality()

In [ ]:
print(model.get_cpds(node="Age"))

In [ ]:
print(model.get_cpds(node="DrivQuality"))

Can you calculate the probability of `DrivQuality` given `Age` for both `Adolescent` and `Senior` values of `Age`? 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()